In [4]:
import numpy as np
import pandas as pd
import random

As mentioned in read me 
Default number of hidden states is 5
Default number is also 5, all the elements in visible sequence is between [0,4]
If you have raw sequence on which HMM has to be applied first get number number of unique hidden states and unique visible states and map states and visible states as follows

In [186]:
sequence=[0,1,2,3,1,1,1,1,2,2,2,2,4,4,4,4,4,1,0,0,0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,3,4,4,4,4,4,4,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,1,1]
sequence=np.array(sequence)
intial=[0.05,0.5,0.3,0.05,0.1]
intial=np.array(intial)

In [187]:
### .............................................................Implementation of hidden markov model.................................................................
## 3 Taks to train HMM
  ## 1) Forward Algorithm
  ## 2) Backward Algorithm
  ## 3) Forward Backward Algorithm

class HMM:
    def __init__(self,sequence,initial_matrix,number_states=5,number_output=5,n_epochs=30):
        self.seq=sequence
        self.tras=np.random.uniform(0,1,(number_states,number_states))  ## Transition Matrix 
        #self.tras=np.zeros(shape=(number_states,number_states))        ## Initialized with random number drawn from uniform (0,1)
        self.emiss=np.random.uniform(0,1,(number_states,number_output)) ## Emission Matrix
        #self.emiss=np.zeros(shape=(number_states,number_output))

        self.iprob=initial_matrix         ## Intial Probabilty matrix (Pie)
        
        self.states=number_states         ## Number of hidden States
        
        self.out=number_output            ## Number of Visible States
        
        self.epochs=n_epochs              ## Number of Iterations default value is 30
        
        self.BDP=np.zeros(shape=(number_states,len(self.seq)))  ## 2d numpy array to store alpha(state,time)
        
        self.FDP=np.zeros(shape=(number_states,len(self.seq)))   ## 2d numpy array to store  Beta(state,time)
        
        self.n_epochs=n_epochs
        
    def forward(self):                                                          
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        maxi=0
        index=0                                                                     
        for j in range(nlength):
            for i in range(nstates):                         
                if j==0:
                    self.FDP[i][j]=self.iprob[i]*self.emiss[i][self.seq[j]]
                else:
                    for k in range(nstates):
                        self.FDP[i][j]+=(self.FDP[k][j-1]*self.tras[k][i]*self.emiss[i][self.seq[j]])
                    
        for j in range(nlength):
            sum1=0.0
            for i in range(nstates):
                sum1+=self.FDP[i][j]
            for i in range(nstates):
                self.FDP[i][j]=self.FDP[i][j]/sum1
        
    ## Dynamic algorithm Space complexity O(states*length of Sequence) Time complexity O(states*states*length of sequence)
    def GiveProb(self):
        nstates=self.states                                                       
        nlength=len(self.seq)
        print(self.FDP.shape)
        for i in range(nstates):
            print(self.FDP[i][nlength-1])
        
    def one(self):
        nstates=self.states                                                       
        nlength=len(self.seq)
        
        for j in range(nlength):
            for i in range(nstates):
                    if(self.FDP[i][j]>1):
                        print("Fraud")
        
    def backward(self):
        ## Algo for calculating B(state,time)
        nstates=self.states                                                 
        nlength=len(self.seq)                                                     
        for j in range(nlength-1,-1,-1):
            for i in range(nstates):
                if(j==nlength-1):
                    self.BDP[i][j]=1.0
                else:
                    for k in range(nstates):
                        self.BDP[i][j]+=(self.tras[i][k]*self.emiss[k][self.seq[j+1]]*self.BDP[k][j+1])
                   
        sum1=0
        for j in range(nlength):
            for i in range(nstates):
                sum1+=self.BDP[i][j]
            for i in range(nstates):
                self.BDP[i][j]=self.BDP[i][j]/sum1
    def not_quite(self,time,curr_state,next_state):
        #self.forward()
        #self.backward()
        ## Algo for calculating mu as per paper
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for i in range(nstates):
            deno+=self.FDP[i][time]*self.BDP[i][time]
        numo=self.FDP[curr_state][time]*self.tras[curr_state][next_state]*self.emiss[next_state][self.seq[time+1]]*self.BDP[next_state][time+1]
        return numo/deno
    
    
    def corrected_transition(self):
        ## Corrected transiton matrix 
        nstates=self.states                                                 
        nlength=len(self.seq)
        
        for i in  range(nstates):
            for j in range(nstates):
                numo=0
                deno=0
                for time in range(nlength-1):
                    numo+=self.not_quite(time,i,j)
                    for state in range(nstates):
                        deno+=self.not_quite(time,i,state)
                self.tras[i][j]=numo/deno
    
    
    def not_quite2(self,time,state):
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for st in range(nstates):
            deno+=self.FDP[st][time]*self.BDP[st][time]
        return self.FDP[state][time]*self.BDP[state][time]/deno
    
    
    def corrected_emmision(self):
        # Algorithm for calculating gamma as per pdf
        nstates=self.states                                                 
        nlength=len(self.seq)
        nout=self.out
        for states in (range(nstates)):
            for out in (range(nout)):
                numo=0
                deno=0
                for time in range(nlength):
                    if(self.seq[time]==out):
                        numo+=self.not_quite2(time,states)
                    deno+=self.not_quite2(time,states)
                self.emiss[states][out]=numo/deno
                
    def display(self):
        print("Transition Matrix")
        print(self.tras)
        print(" ")
        print(" ")
        print("Emmision Matrix")
        print(self.emiss)
            
    def train(self):
        for epoch in range(self.n_epochs):               ## Iterating fot number of epochs
            print("EPOCH NO : {} TRANING".format(epoch))
            self.forward()                               ## calculate corrected alpha 
            self.backward()                              ## calculate corrected Beta
            self.corrected_transition()                  ## Corrected Transition matrix
            self.corrected_emmision()                    ## Corrected Emission matrix
            print("Transition Matrix")
            print(self.tras)
            print("Emmission Matrix")
            print(self.emiss)
            print("Traning Epoch Done")
            print(" ")
            print(" ")

In [188]:
# deno+=self.not_quite(time,i,states)

In [189]:
hm=HMM(sequence,intial,n_epochs=50)

In [190]:
hm.one()

In [191]:
hm.train()

EPOCH NO : 0 TRANING
Transition Matrix
[[0.4741985  0.06532616 0.09863843 0.51473317 0.00325948]
 [0.13458487 0.46379499 0.22299997 0.24065888 0.21918742]
 [0.29857849 0.50134037 0.14018845 0.02368474 0.12080468]
 [0.39417236 0.22676663 0.18283008 0.04401028 0.21904476]
 [0.19702796 0.385104   0.04080632 0.1640585  0.49893791]]
Emmission Matrix
[[0.01337212 0.36878384 0.26139782 0.00449353 0.35195269]
 [0.29549879 0.34341394 0.08963558 0.21198697 0.05946472]
 [0.41254777 0.11554167 0.12973211 0.12551027 0.21666817]
 [0.10745235 0.27763849 0.26218958 0.13055172 0.22216786]
 [0.11329735 0.05775769 0.10004064 0.59126249 0.13764183]]
Traning Epoch Done
 
 
EPOCH NO : 1 TRANING
Transition Matrix
[[0.54752081 0.04490624 0.05051472 0.33331499 0.00225465]
 [0.09478095 0.44985931 0.13171116 0.13749299 0.25414798]
 [0.23632946 0.55520936 0.10872697 0.0136714  0.115478  ]
 [0.40378227 0.21122334 0.11623127 0.03163978 0.25098536]
 [0.09938282 0.20938934 0.01779913 0.07855844 0.75861907]]
Emmission

Transition Matrix
[[7.11495420e-01 4.79348126e-02 4.07448858e-02 1.88032022e-01
  1.15540265e-05]
 [5.47233473e-02 9.06576461e-01 9.88007350e-03 3.50251248e-03
  9.33996057e-07]
 [6.94228209e-04 7.27019605e-01 2.90442472e-01 2.15269189e-05
  1.46179788e-05]
 [2.47345797e-01 5.09868860e-02 2.71000487e-02 3.83076503e-01
  2.56335825e-01]
 [1.61740623e-01 3.12953207e-08 2.19391119e-09 5.11402630e-05
  8.31806893e-01]]
Emmission Matrix
[[1.48929083e-04 2.03932777e-01 2.62312835e-01 2.55184670e-04
  5.33350274e-01]
 [4.04015365e-01 5.71204327e-01 8.98443882e-03 9.78926295e-03
  6.00660568e-03]
 [7.15712429e-01 7.21187482e-02 1.73619571e-02 3.69427494e-02
  1.57864117e-01]
 [7.84104778e-03 1.06078562e-01 7.18078877e-01 2.49270632e-02
  1.43074450e-01]
 [3.33170412e-04 4.43548631e-03 1.14457230e-01 8.56758783e-01
  2.40153296e-02]]
Traning Epoch Done
 
 
EPOCH NO : 12 TRANING
Transition Matrix
[[6.76684511e-01 7.64014034e-02 7.42058455e-02 1.62665198e-01
  3.64381379e-06]
 [7.86244701e-02 8.7

Transition Matrix
[[8.09025660e-01 7.40063147e-02 1.17443592e-01 4.67497453e-14
  4.78427480e-25]
 [1.95381457e-16 7.79583102e-01 1.77565297e-07 2.28569396e-01
  1.25276751e-30]
 [2.00438689e-21 1.56888949e-01 8.47483108e-01 9.69976808e-27
  1.73406097e-33]
 [4.51198673e-02 2.71463761e-11 9.98702011e-15 7.72999859e-01
  1.90651550e-01]
 [1.83611036e-01 5.22054135e-11 2.28192442e-29 7.25140818e-25
  8.18464134e-01]]
Emmission Matrix
[[7.90477635e-05 6.89494525e-02 3.05731157e-03 2.06999496e-04
  9.27707189e-01]
 [1.11911443e-03 9.47055231e-01 1.22408282e-02 3.95844921e-02
  3.34299557e-07]
 [9.66631458e-01 2.67987883e-02 1.01684150e-03 5.49130640e-03
  6.16052568e-05]
 [5.25297849e-07 5.91786402e-03 9.92677261e-01 8.67249891e-04
  5.37099763e-04]
 [1.49724161e-08 3.46240032e-04 6.54600871e-03 9.92364467e-01
  7.43269135e-04]]
Traning Epoch Done
 
 
EPOCH NO : 24 TRANING
Transition Matrix
[[8.07396410e-01 7.44904165e-02 1.18369173e-01 2.77482558e-16
  8.83286788e-28]
 [1.80641023e-18 7.7

Transition Matrix
[[8.06058703e-01 6.67381746e-02 1.26274213e-01 7.41774510e-49
  8.28394926e-65]
 [1.09650757e-41 8.40271000e-01 1.32366144e-12 1.59690014e-01
  5.88799594e-69]
 [3.06448485e-47 1.80499928e-01 8.20028509e-01 2.99657193e-58
  2.35411537e-84]
 [7.31755690e-02 1.02964324e-19 1.29651640e-41 7.37504490e-01
  1.85996927e-01]
 [1.88759378e-01 7.23693041e-07 6.33664159e-50 6.00675127e-65
  8.08570524e-01]]
Emmission Matrix
[[1.95093058e-05 4.91025619e-02 5.74352406e-04 3.97103167e-05
  9.50263866e-01]
 [3.17761232e-08 9.05023047e-01 4.92910926e-02 4.56858282e-02
  2.46610557e-11]
 [9.55057282e-01 4.39940460e-02 1.90945266e-04 7.57725152e-04
  1.46472097e-09]
 [3.10409757e-10 1.51020745e-03 9.98385500e-01 1.49879720e-05
  8.93037906e-05]
 [2.42134612e-10 4.82331188e-05 1.79688949e-04 9.99690534e-01
  8.15433026e-05]]
Traning Epoch Done
 
 
EPOCH NO : 36 TRANING
Transition Matrix
[[8.08391430e-01 6.68233884e-02 1.24327356e-01 2.01171929e-52
  8.38260214e-69]
 [6.88971342e-44 8.3

Transition Matrix
[[8.01410226e-001 7.34500860e-002 1.25384774e-001 1.31735944e-096
  9.96104444e-119]
 [6.68609486e-069 8.34477414e-001 2.05090273e-018 1.65419622e-001
  1.07453296e-110]
 [2.17797378e-075 1.84428813e-001 8.16283092e-001 5.05872086e-096
  1.44882889e-153]
 [6.36064160e-002 1.17925827e-031 2.41107422e-080 7.42710412e-001
  1.93658966e-001]
 [1.81953480e-001 9.47110747e-006 4.23608259e-071 3.51331068e-122
  8.18024230e-001]]
Emmission Matrix
[[4.84307059e-06 3.30856027e-02 4.94776928e-05 3.23377865e-06
  9.66856843e-01]
 [2.28887520e-12 8.96606982e-01 5.61974920e-02 4.71955258e-02
  3.36729386e-13]
 [9.34711592e-01 6.52182797e-02 1.77078721e-05 5.24208451e-05
  3.93397964e-14]
 [1.62689990e-11 2.76452679e-04 9.99709901e-01 4.67009099e-07
  1.31792169e-05]
 [1.42601610e-11 7.97139676e-06 5.64505968e-06 9.99974435e-01
  1.19485602e-05]]
Traning Epoch Done
 
 
EPOCH NO : 48 TRANING
Transition Matrix
[[8.00298785e-001 7.32978256e-002 1.26644527e-001 4.78554450e-101
  8.79923

In [192]:
hm.display()

Transition Matrix
[[7.99444902e-001 7.29889960e-002 1.27776611e-001 1.48543184e-105
  6.46028665e-129]
 [1.29274246e-073 8.35476150e-001 2.25242206e-019 1.64446147e-001
  8.86742545e-118]
 [2.96554223e-080 1.81533707e-001 8.19694228e-001 1.08347074e-102
  2.20571270e-166]
 [6.40899988e-002 9.67061247e-034 1.46554192e-087 7.42204156e-001
  1.93461173e-001]
 [1.82238498e-001 9.67518248e-006 1.32215670e-074 4.89235664e-133
  8.17610128e-001]]
 
 
Emmision Matrix
[[3.92784335e-06 3.06578470e-02 3.45147410e-05 2.23143114e-06
  9.69301479e-01]
 [5.18589502e-13 8.96218460e-01 5.65028899e-02 4.72786500e-02
  1.76445290e-13]
 [9.31796288e-01 6.81560525e-02 1.24671907e-05 3.51919827e-05
  7.19319157e-15]
 [1.04256619e-11 2.11493578e-04 9.99778399e-01 2.74559888e-07
  9.83330710e-06]
 [9.16206088e-12 6.10242891e-06 3.42030623e-06 9.99981543e-01
  8.93429885e-06]]
